# CarbonTracker model training
This notebook outlines the entire workflow to load and preprocess the following data sets, to be able to train a ML model on CarbonTracker's carbon flux:

- CarbonTracker
- ERA5 (monthly)
- ERA5-land (monthly)
- SPEI (monthly)
- MODIS (monthly)
- Biomass (yearly)
- Copernicus Landcover (yearly)

First follow the data downloading and config setup instuctions.

If you run this notebook on Surf Research Cloud, you shouldn't need to do this anymore.

In [3]:
import datetime
from pathlib import Path

import xarray as xr
from dask.distributed import Client

import excited_workflow
from excited_workflow import train_carbontracker_model
from excited_workflow.source_datasets import datasets


client = Client()

Define the paths for the carbon tracker dataset, the regions dataset and create output directory, as well as define the datasets that need to be included, the input variables (x_keys) for the model and the target variable (y_key).

In [4]:
cb_file= Path("/data/volume_2/EXCITED_prepped_data/CT2022.flux1x1-monthly.nc")
regions_file = Path("/data/volume_2/EXCITED_prepped_data/regions.nc")
output_path = Path.home()

time = datetime.datetime.now().strftime("%Y-%m-%d_%H_%M")
output_dir = output_path / f"carbon_tracker-{time}"
output_dir.mkdir(parents=True, exist_ok=True)

desired_data = [
    "biomass",
    "spei",
    "modis",
    "era5_monthly",
    "era5_land_monthly",
    "copernicus_landcover"
]

x_keys = ["d2m", "mslhf", "msshf", "ssr", "str", "t2m", "spei", "NIRv", "skt",
            "stl1", "swvl1", "lccs_class"]
y_key = "bio_flux_opt"

Merge the desired datasets into a single xr.Dataset with the same dimensions as the carbon tracker dataset. 

In [5]:
ds_cb = xr.open_dataset(cb_file)
ds_cb = excited_workflow.utils.convert_timestamps(ds_cb)
ds_input = xr.merge(
    [
        datasets[name].load(freq="monthly", target_grid=ds_cb)
        for name in desired_data
    ]
)

To limit the analyis to Transcom region 2 (North America) we require the `regions.nc` file:

In [6]:
ds_na, df_na = train_carbontracker_model.mask_region(regions_file, cb_file, ds_input, 2, y_key)

In [7]:
ds_na

<xarray.Dataset>
Dimensions:           (time: 240, latitude: 180, longitude: 360)
Coordinates:
  * longitude         (longitude) float64 -179.5 -178.5 -177.5 ... 178.5 179.5
  * latitude          (latitude) float64 -89.5 -88.5 -87.5 ... 87.5 88.5 89.5
  * time              (time) datetime64[ns] 2000-01-01 2000-02-01 ... 2019-12-01
Data variables: (12/26)
    bio_flux_opt      (time, latitude, longitude) float64 nan nan ... nan nan
    transcom_regions  (latitude, longitude, time) float64 nan nan ... nan nan
    biomass           (time, latitude, longitude) float64 nan nan ... nan nan
    spei              (time, latitude, longitude) float64 nan nan ... nan nan
    NDVI              (time, latitude, longitude) float64 nan nan ... nan nan
    NIRv              (time, latitude, longitude) float64 nan nan ... nan nan
    ...                ...
    stl4              (time, latitude, longitude) float64 nan nan ... nan nan
    swvl1             (time, latitude, longitude) float64 nan nan ... nan nan
    swvl2             (time, latitude, longitude) float64 nan nan ... nan nan
    swvl3             (time, latitude, longitude) float64 nan nan ... nan nan
    swvl4             (time, latitude, longitude) float64 nan nan ... nan nan
    lccs_class        (time, latitude, longitude) float32 nan nan ... nan nan
Attributes:
    averaging_period_length_hours:  744
    email:                          carbontracker.team@noaa.gov
    url:                            http://carbontracker.noaa.gov
    institution:                    NOAA Earth System Research Laboratory
    Conventions:                    CF-1.5
    history:                        Time-stamp: <Orion-login-1.HPC.MsState.Ed...
    NCO:                            netCDF Operators version 4.9.3 (Homepage ...
    version:                        CT2022 1x1 3-hourly fluxes as of 2023-01-...

Validate the model by splitting the dataset into 5 groups. Train the model over 4 groups and predict over the remaining group iteratively. Output rmse netcdfs and scatterplots are stored in the output directory.

In [8]:
train_carbontracker_model.validate_model(ds_na, 5, x_keys, y_key, output_dir)

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
lightgbm,Light Gradient Boosting Machine,0.00000,0.00000,0.00000,0.77680,0.00000,3.73310,1.17000


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
lightgbm,Light Gradient Boosting Machine,0.00000,0.00000,0.00000,0.76350,0.00000,8.80630,1.12000


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
lightgbm,Light Gradient Boosting Machine,0.00000,0.00000,0.00000,0.76770,0.00000,4.25830,1.12000


2024-02-28 10:46:43,177 - distributed.utils_perf - WARNING - full garbage collections took 15% CPU time recently (threshold: 10%)
2024-02-28 10:46:43,628 - distributed.utils_perf - WARNING - full garbage collections took 19% CPU time recently (threshold: 10%)


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
lightgbm,Light Gradient Boosting Machine,0.00000,0.00000,0.00000,0.76360,0.00000,6.42310,1.13000


...

2024-02-28 10:46:52,032 - distributed.utils_perf - WARNING - full garbage collections took 23% CPU time recently (threshold: 10%)
2024-02-28 10:46:52,557 - distributed.utils_perf - WARNING - full garbage collections took 23% CPU time recently (threshold: 10%)
2024-02-28 10:46:53,013 - distributed.utils_perf - WARNING - full garbage collections took 24% CPU time recently (threshold: 10%)


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
lightgbm,Light Gradient Boosting Machine,0.00000,0.00000,0.00000,0.75680,0.00000,4.29950,1.12000


2024-02-28 10:46:57,656 - distributed.utils_perf - WARNING - full garbage collections took 21% CPU time recently (threshold: 10%)
2024-02-28 10:46:58,000 - distributed.utils_perf - WARNING - full garbage collections took 21% CPU time recently (threshold: 10%)
2024-02-28 10:46:58,469 - distributed.utils_perf - WARNING - full garbage collections took 21% CPU time recently (threshold: 10%)
2024-02-28 10:46:59,151 - distributed.utils_perf - WARNING - full garbage collections took 24% CPU time recently (threshold: 10%)
2024-02-28 10:46:59,377 - distributed.utils_perf - WARNING - full garbage collections took 24% CPU time recently (threshold: 10%)
2024-02-28 10:47:00,830 - distributed.utils_perf - WARNING - full garbage collections took 23% CPU time recently (threshold: 10%)


2024-02-28 10:47:01,221 - distributed.utils_perf - WARNING - full garbage collections took 23% CPU time recently (threshold: 10%)
2024-02-28 10:47:01,721 - distributed.utils_perf - WARNING - full garbage collections took 23% CPU time recently (threshold: 10%)


<Figure size 600x600 with 0 Axes>

Train the model over the entire dataset.

In [9]:
pycs, model = train_carbontracker_model.train_model(df_na, x_keys, y_key)

2024-02-28 10:47:04,433 - distributed.utils_perf - WARNING - full garbage collections took 22% CPU time recently (threshold: 10%)
2024-02-28 10:47:07,270 - distributed.utils_perf - WARNING - full garbage collections took 20% CPU time recently (threshold: 10%)


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
lightgbm,Light Gradient Boosting Machine,0.00000,0.00000,0.00000,0.76450,0.00000,7.44970,1.38000


2024-02-28 10:47:09,911 - distributed.utils_perf - WARNING - full garbage collections took 19% CPU time recently (threshold: 10%)
2024-02-28 10:47:10,304 - distributed.utils_perf - WARNING - full garbage collections took 19% CPU time recently (threshold: 10%)
2024-02-28 10:47:10,810 - distributed.utils_perf - WARNING - full garbage collections took 22% CPU time recently (threshold: 10%)
2024-02-28 10:47:11,586 - distributed.utils_perf - WARNING - full garbage collections took 22% CPU time recently (threshold: 10%)
2024-02-28 10:47:11,828 - distributed.utils_perf - WARNING - full garbage collections took 22% CPU time recently (threshold: 10%)
2024-02-28 10:47:13,577 - distributed.utils_perf - WARNING - full garbage collections took 21% CPU time recently (threshold: 10%)


Save the model to ONNX in the output directory. 

In [10]:
train_carbontracker_model.save_model(pycs, model, output_dir)

The maximum opset needed by this model is only 8.


Create dataframe to run the model with. 

In [11]:
df = df_na[x_keys]
df.head()

d2m       mslhf      msshf         ssr  \
time       latitude longitude                                                  
2000-08-01 16.5     -97.5      291.160736 -123.552444 -29.256828  17632392.0   
                    -96.5      287.451385  -98.636139 -44.350342  17875046.0   
                    -95.5      292.602661  -88.933502 -76.770813  18379498.0   
           17.5     -100.5     290.568329 -118.364532 -51.205177  17707054.0   
                    -99.5      290.078156 -119.800644 -42.374344  17862124.0   

                                     str         t2m      spei      NIRv  \
time       latitude longitude                                              
2000-08-01 16.5     -97.5     -4539167.0  294.455078  0.402637  0.264352   
                    -96.5     -5688396.0  291.743347  0.859432  0.186314   
                    -95.5     -4277693.5  298.020874 -0.002091  0.245570   
           17.5     -100.5    -3109625.0  293.402252  0.364373  0.270905   
                    -99.5     -3914028.0  293.219940  0.007593  0.249498   

                                      skt        stl1     swvl1  lccs_class  
time       latitude longitude                                                
2000-08-01 16.5     -97.5      293.800140  294.488708  0.405656        90.0  
                    -96.5      291.346344  292.329987  0.441099       120.0  
                    -95.5      298.976379  299.503906  0.340303        60.0  
           17.5     -100.5     293.096253  293.790924  0.412808        90.0  
                    -99.5      291.973236  292.737579  0.406332       120.0

Open model and run it over the dataframe to check it was saved correctly. 

In [12]:
from onnxruntime import InferenceSession

with open(output_dir / "lightgbm.onnx", "rb") as f:
    model = f.read()

sess = InferenceSession(model)
predictions_onnx = sess.run(None, {'X': df.to_numpy()})[0]

In [13]:
predictions_onnx

array([[-1.1447313e-06],
       [-6.4269062e-07],
       [-4.5004401e-07],
       ...,
       [ 4.5509165e-07],
       [ 4.7937402e-07],
       [ 1.3048043e-07]], dtype=float32)